In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision.transforms import ToTensor, Lambda

from PIL import Image, ImageDraw
import pandas as pd

f"/nsls2/users/maire1/sim_data/sim{i}.csv"

In [ ]:
def set_rand_seed(num):
    '''
    Creates random seed so that randomness is repeatable. Initialized first to set all randomness
    '''
    np.random.seed(num)
    
def set_k():
    """
    Uses random seed to randomly set k - changes size of the speckles
    """
    return(np.random.randint(5,100))

def make_mesh(window_size):
    """
    Creates 2D meshgrid for calculating all the functions.
    
    Inputs: 
        Image size (assumes square image)
    Outputs: 
        Meshgrid according to the image dimensions
    """
    x = np.arange(window_size)
    y = np.arange(window_size)
    xx, yy = np.meshgrid(x,y)
    return xx,yy

def object_shape(x, y, x_center, y_center, sigma):
    """
    Defines the shape of the average intensity, can be changed for any other form.
    
    Inputs: 
        x & y: arrays of values
        x_center & y_center: center of shape
    Outputs: 
        Circle 
    """
    return np.exp(-((x-x_center)**2 + (y-y_center)**2)/2/sigma)
           
def create_object(window_size, sigma = 5e4):
    """
    Calculates the average intensity on the defined mesh (i.e. on the CDD detector).
    """
    xx, yy = make_mesh(window_size)
    z = object_shape(xx, yy, 350,350, sigma)
    return z

def circle(r):
    return r<1

def bandpass(p,q,r0, n):
    """
    Defines the functional form of the lens. Lens is needed to make a point source.
    """
    return circle(np.sqrt((p-n/2)**2+(q-n/2)**2)/r0)

def make_bandpass(window_size, r0):
    """
    Calculates the lens band pass form on the defined mesh grid.
    """
    xx, yy = make_mesh(window_size)
    band_pass = bandpass(xx, yy, r0, window_size)
    return band_pass

In [ ]:
def make_speckles(n, r0):
    """
    Creates speckles
    
    Inputs: 
        n: window size (assumes square image)
        r0: n/k
    Outputs: Speckle array
    """
    # random phasors
    rand_field = np.random.rand(n,n) * 1j
    rand_field = np.exp(2*np.pi*rand_field)

    # object intensity in the absence of the speckle        
    object_intensity = create_object(window_size = n)

    # multiply the amplitude distribution by random phasors
    scattered_field = np.sqrt(object_intensity)*rand_field

    # circular function of the lens
    band_pass = make_bandpass(n, r0)

    # field transmitted by the lens
    pupil_field = band_pass*np.fft.fft2(scattered_field)


    image_field = np.fft.ifft2(pupil_field)
    image_intensity = np.abs(image_field)**2
    
    return(image_intensity)


def make_beam_streaks(window_size):
    """
    Creates a random number of triangles to simulate beam streaks. Coordinates are randomly generated around the window.
    Returns: np.array of triangles. For masking purposes, value is set to 1
    """
    
    blank = Image.new('L', (window_size,window_size),0)
    #Create random number of beam streaks between 0 and 4
    num_beam_streaks = np.random.randint(0,4)
    
    for i in range(num_beam_streaks):
        #Generating three random vertices
        rand_points = window_size * (np.random.rand(2,2))
        points = np.append(rand_points, [[window_size*np.random.random_sample(), np.sqrt(rand_points[0,1]**2 + rand_points[1,1]**2)]], axis=0)
        
        #Changing format of data points. Yes, there is likely an easier method to do this rather than hardcoding
        a = [(points[0,0],points[0,1]), (points[1,0],points[1,1]), (points[2,0],points[2,1])]
        ImageDraw.Draw(blank).polygon(a, outline=1, fill = 1)
    
    return(np.asarray(blank))

def make_beam_stop(n):
    """
    Creates coordinates and image for beam stop
    Returns: Beam stop as array. For masking purposes, value is set to 2
    """
    rand_points = n * (np.random.rand(2,1))
    rect_coord = [(rand_points[0],rand_points[1]), (rand_points[0],rand_points[1]+1/8*n), 
                  (rand_points[0]+1/8*n,rand_points[1]+1/8*n), (rand_points[0]+1/8*n,rand_points[1])]
    #rect_coord = [(1/8*n, 1/8*n), (1/8*n, 1/4*n), (1/4*n, 1/4*n), (1/4*n, 1/8*n)]
    r = Image.new('L', (n,n), 0)
    ImageDraw.Draw(r).polygon(rect_coord, outline=2, fill=2)
    
    r = np.asarray(r)
    r = np.where(r==2, np.nan, r) 

    return(r)

def add_noise(window_size):
    """
    Adds random noise to image.
    """
    random_array = np.random.rand(window_size,window_size)
    return(random_array)


def save(sample,i, sample_name, sample_address):
    """
    Saves file in sim_data folder with iterating number to correspond to number of samples desired as pytorch tensors
    """
    input_tensor = torch.Tensor(sample['input'])
    target_tensor = torch.Tensor(sample['target'])

    torch.save({"input": input_tensor, "target": target_tensor}, f"/nsls2/users/maire1/sim_data/sim{i}.pt")
    
    #pd.Series(sample).to_frame().to_csv(f"/Users/morganaire/Documents/BNL/speckle_sim/sim_data/sim{i}.csv")
    #np.save(f'sim{i}', sample)
    sample_name.append(f"sim{i}.pt")
    sample_address.append(f"/nsls2/users/maire1/sim_data/sim{i}.pt")
    
    return(sample_name, sample_address)

def save_total_data(name, address):
    """
    Saving titles and sample addresses into a separate csv file for use in the neural network.
    
    TODO: Save as a torch tensor in the future. 
    """
    
    d = {'sample': name, 'address': address}
    df = pd.DataFrame(data=d)
    
    #torch.save(df)
    df.to_csv('/nsls2/users/maire1/sim_data/sim_address.csv', index = False)
    print("All samples completed. Data saved.")
    
def rescale(intensity, noise):
    '''
    This function rescales the intensity so that the speckles are always visible above the noise.
    This should also add some complexity to the simulation because speckle data is closer in intensity to beam streaks (beam streaks are around 1, speckles are now set to be > noise (~1))
    Returns: new intensity function
    '''
    while intensity.max() < noise:
        intensity = intensity * 5
    return intensity
    
def create_a_sample(rand_seed_num, n):
    """
    Does everything necessary to create both a full sample along with masks. 
    Reutrns: Dictionary with imput and target to be saved.
    """
    #Set initial params
    set_rand_seed(rand_seed_num)
    
    k = set_k()
    r0 = n/k
    
    #Creating all pieces of image
    image_intensity = make_speckles(n, r0)
    size = image_intensity.shape
    mask = np.zeros(size)
    Polygon = make_beam_streaks(n)
    beam_stop = make_beam_stop(n)
    
    img_intensity_rescaled = rescale(image_intensity,add_noise(n).max()) #rescale intensity value so that it is always visible and greater than the noise
    
    # Creating total image and setting beam stop values to zero
    new_image = img_intensity_rescaled + Polygon + beam_stop + add_noise(n)
    new_image = np.nan_to_num(new_image, nan=0)
    
    # Creating mask and setting beam stop values to zero
    mask = Polygon + mask + beam_stop
    mask = np.nan_to_num(mask, nan=2)

    #plt.imshow(new_image)
    #plt.show()
    sample_dict = {'input':new_image, 'target': mask}
    
    return (sample_dict)

In [ ]:
'''MAIN'''
samples = 1000
window_size = 256

num_samples = np.arange(0,samples)
sn = []
sa = []

for j in (num_samples):
    x = create_a_sample(j, window_size)
    sn, sa = save(x,j, sn, sa)
save_total_data(sn, sa)